# Ejercicio 1

Codifique el algoritmo de entrenamiento para una red neuronal con RBF y pruebe su implementación en la resolución del problema `xor`, empleando los datos de la guía anterior.
Diseñe una red RBF para resolver el problema Iris (`irisbin.csv`), considerando una cantidad de parámetros similar a la red MLP de la guía anterior. 
Luego compare la velocidad de entrenamiento y el porcentaje final de clasificación obtenido con ambas arquitecturas.